Si nunca ejecutaste el codigo, es mejor que por terminal ejecutes el siguiente comando para descargar las librerias:

In [ ]:
# Este codigo ------->   pip install pandas pyreadr numpy matplotlib seaborn

In [ ]:
#Codigo de importaciones
#Importaciones necesarias para que funcione el codigo
import pandas as pd
import pyreadr
import os
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
#Codigo de creación de carpetas
# Nombre de la carpeta donde antes de ejecutar el script se guardan los datos CaliBRANDO2024.dta
carpeta = "insumos"
if not os.path.exists(carpeta):
    os.makedirs(carpeta)

carpeta = "insumos/data"
if not os.path.exists(carpeta):
    os.makedirs(carpeta)

carpeta = "insumos/marco"
if not os.path.exists(carpeta):
    os.makedirs(carpeta)

# Nombre de la carpeta donde se guardaran los datos
carpeta = "outputs"
if not os.path.exists(carpeta):
    os.makedirs(carpeta)

carpeta = "outputs/tablasFrecuencia"
if not os.path.exists(carpeta):
    os.makedirs(carpeta)

In [ ]:
#Se lee la base de datos
df = pd.read_stata("insumos/data/Panel 2023_ 1.dta")  # Leer el archivo .dta

In [ ]:
#Mostrar base de datos
df

In [ ]:
#Se muestran los labels de las variables
# Cargar el archivo con un iterador
data_reader = pd.read_stata("insumos/data/Panel 2023_ 1.dta", iterator=True)

# Extraer las etiquetas de las variables
variable_labels = data_reader.variable_labels()

# Leer todo el archivo en un DataFrame
df = data_reader.read()

# Agregar las etiquetas manualmente como un atributo del DataFrame
df.attrs["variable_labels"] = variable_labels

# Ver las etiquetas
print(df.attrs["variable_labels"])


In [ ]:
#Se carga el marco instrumental
#Se lee el excel y se almacena como dataFrame (df), en este caso el Marco instrumental
marco_instrumental = pd.read_excel("insumos/marco/Marco Instrumental_CaliBRANDO 2024.xlsx", skiprows=1)#Se salta la primera fila del excel ya que no tiene informacion

# Eliminar las primeras N columnas (por ejemplo, las primeras 1)
marco_instrumental = marco_instrumental.iloc[:, 1:]  # Cambia el número según cuántas columnas quieras omitir

marco_instrumental

In [ ]:
#Se ve información por año
# Contar valores y convertir a DataFrame
df_counts = df["P01000801"].value_counts().reset_index()

# Renombrar columnas
df_counts.columns = ["P01000801", "Numero de encuestas"]

# Convertir los años a enteros
df_counts["P01000801"] = df_counts["P01000801"].astype(int)

# Ordenar por P01000801 en orden descendente
df_counts = df_counts.sort_values(by="P01000801", ascending=False).reset_index(drop=True)

# Filtrar para mostrar solo de 2023 a 2015
df_counts = df_counts[df_counts["P01000801"].between(2015, 2023)]

# Mostrar como tabla
df_counts


In [ ]:
# Obtener las Masterclas únicas
masterclas_unicas = marco_instrumental["Masterclas"].dropna().unique().tolist()

# Mostrar resultado en formato de lista vertical
print("Masterclas únicas:")
i=0
for masterclas in masterclas_unicas:
    print(f"({i}):",masterclas,".")
    i=i+1


In [ ]:
# Seleccionar una masterclas (Que no sea 0 que es logistica)
num_masterclas_seleccionada = 1
masterclas_seleccionada = masterclas_unicas[num_masterclas_seleccionada]
print("Masterclas seleccionada:", masterclas_seleccionada)


#print("Seleccione una masterclas por su numero:")
#masterclas_seleccionada = masterclas_unicas[int(input())]  

In [ ]:
# Agrupar los códigos por "Masterclas"
grupos_masterclas = marco_instrumental.groupby("Masterclas")["Code"].apply(list).to_dict()

# Función para obtener códigos de una Masterclas específica
def obtener_codigos(masterclas):
    return grupos_masterclas.get(masterclas, [])

# Ejemplo de uso: obtener códigos para una "Masterclas" específica
codigos = obtener_codigos(masterclas_seleccionada)

print(f"Códigos para {masterclas_seleccionada}: {codigos}")


In [ ]:
# Ver las etiquetas de todo el df
print(df.attrs["variable_labels"])
variable_labels=df.attrs["variable_labels"]

In [ ]:
#Generar las tablas de los codigos seleccionados
# Definir lista de años
years = [2015, 2016, 2017, 2018, 2019, 2021, 2022, 2023]
codigos_interes = codigos  # Lista de preguntas/códigos

# Filtrar `df` para que solo contenga las columnas de interés
df_filtrado = df[["P01000801"] + [col for col in df.columns if col in codigos_interes]].copy()

# Convertir la columna de años a enteros
df_filtrado["P01000801"] = df_filtrado["P01000801"].astype(int)

# Asegurar que no haya valores nulos en el año
df_filtrado = df_filtrado.dropna(subset=["P01000801"])

# Diccionario para almacenar las tablas de cada código
tablas_resultado = {}

# Iterar sobre cada código en codigos_interes
for col in codigos_interes:
    if col in df_filtrado.columns:
        # Obtener tipo de variable
        tipo_variable = df_filtrado[col].dtype
        
        # Obtener etiqueta del código
        etiqueta = variable_labels.get(col, "Sin etiqueta disponible")

        # Calcular la frecuencia relativa (%) por año
        tabla_frecuencia = (
            df_filtrado.groupby("P01000801")[col]
            .value_counts(normalize=True)
            .unstack()
            .fillna(0) * 100
        )

        # Redondear valores a un decimal
        tabla_frecuencia = tabla_frecuencia.round(1)

        # Asegurar que todos los años están en la tabla
        tabla_frecuencia = tabla_frecuencia.reindex(years).fillna("Sin datos")

        # Renombrar índice
        tabla_frecuencia.index.name = "Año"

        # Guardar la tabla en el diccionario
        tablas_resultado[col] = tabla_frecuencia

        # Imprimir la tabla con formato
        print("------------------------------------------------------------------------------------------")
        print(f"\n Tabla de Porcentajes para {col}")
        print(f"🔹 Etiqueta: {etiqueta}")
        print(f"🔹 Tipo de Variable: {tipo_variable}")
        print(tabla_frecuencia.to_string(index=True))

# Guardar todas las tablas en un archivo Excel
with pd.ExcelWriter("outputs/tablasFrecuencia/tablas_frecuencia_porcentaje.xlsx") as writer:
    for col, tabla in tablas_resultado.items():
        tabla.to_excel(writer, sheet_name=col)


In [ ]:
#Selecciona un año para ver la información
year=2021
yearMarco="Var2021"

In [ ]:
# Filtrar los valores donde var2021 no es NaN
filtered_codes_forYear = marco_instrumental.loc[marco_instrumental[yearMarco].notna(), "Code"]

# Mostrar los códigos filtrados
print(filtered_codes_forYear.tolist())  # Para obtener una lista de valores
list_filtered_codes_forYear = filtered_codes_forYear.tolist()
print(len(list_filtered_codes_forYear))  # Para obtener el número de valores

In [ ]:
#Informacion del marco instrumental
print(marco_instrumental.columns)

In [ ]:
# No necesario
# Seleccionar solo las columnas deseadas
#columnas_deseadas = ["Code", "Var2023", "Label", "0", "1", "2" ,"3","4","5","6","7","8","9","10","11","12","13","14","15"]
#df_seleccionado = marco_instrumental[columnas_deseadas]

# Mostrar las primeras filas para verificar
#print(df_seleccionado.head())

In [ ]:
#Se filtra la df por el año seleccionado
dataOfTheYear=df[df["P01000801"] == year]

In [ ]:
dataOfTheYear["P01000801"].value_counts()

print("La cantidad de encuestas en el",year," son : ", dataOfTheYear["P01000801"].value_counts().sum())

In [ ]:
# Ver las etiquetas
print(dataOfTheYear.attrs["variable_labels"])
variable_labels=dataOfTheYear.attrs["variable_labels"]

In [ ]:
# Crear un diccionario para almacenar la informacion de las variables
column_info = {}

for col in dataOfTheYear.columns:
    unique_values = dataOfTheYear[col].nunique()  # Cantidad de valores únicos
    dtype = dataOfTheYear[col].dtype  # Tipo de datos exacto

    # Clasificar el tipo de variable
    if pd.api.types.is_numeric_dtype(dataOfTheYear[col]):
        var_type = "float" if pd.api.types.is_float_dtype(dataOfTheYear[col]) else "int"
    elif pd.api.types.is_string_dtype(dataOfTheYear[col]):
        var_type = "string"
    elif pd.api.types.is_categorical_dtype(dataOfTheYear[col]):
        var_type = "categorical"
    elif pd.api.types.is_datetime64_any_dtype(dataOfTheYear[col]):
        var_type = "datetime"
    else:
        var_type = "other"

    column_info[col] = {"unique_values": unique_values, "dtype": dtype, "variable_type": var_type}

# Convertir a un DataFrame
column_summary = pd.DataFrame.from_dict(column_info, orient='index')

# Guardar en CSV para consulta posterior
#column_summary.to_csv("outputs/dataOfTheYear_column_summary.csv")

# Mostrar el resumen
print(column_summary)


In [ ]:
#Tipos de dato de las variables
column_summary["variable_type"].unique()


In [ ]:
#Columnas de la base de datos
print(dataOfTheYear.columns)

In [ ]:
#Todos los valores unicos de las columnas
for col in dataOfTheYear.columns:
    print(f"{col}: {df[col].unique()}")

In [ ]:
# =====================
# FUNCIONES AUXILIARES
# =====================
def get_variable_label(df, column):
    """Devuelve la etiqueta de una variable si está almacenada en los atributos del DataFrame."""
    return df.attrs.get("variable_labels", {}).get(column, f"Sin etiqueta para '{column}'")

def describe_variable(df, column):
    """Muestra estadísticas descriptivas de una variable."""
    if column not in df or df[column].isna().all():
        return False  # Indica que la variable está vacía
    
    print(f"\nResumen de '{column}' ({get_variable_label(df, column)})")
    print("-" * 50)
    print(df[column].describe())
    
    return True  # Indica que la variable tiene datos

def frequency_table(df, column, max_categories=100):
    """Genera una tabla de frecuencias y etiqueta de la variable, limitando la cantidad mostrada."""
    if column not in df or df[column].dropna().empty:
        return False  # Indica que la columna está vacía
    
    value_counts = df[column].value_counts(dropna=False)
    
    if value_counts.empty:
        return False
    
    freq_table = value_counts.reset_index()
    freq_table.columns = [column, "Frecuencia"]
    freq_table["Porcentaje"] = (freq_table["Frecuencia"] / freq_table["Frecuencia"].sum()) * 100
    freq_table["Etiqueta"] = freq_table[column].map(lambda x: get_variable_label(df, column))

    total_categories = len(freq_table)
    
    print(f"\nTabla de Frecuencias para '{column}' ({get_variable_label(df, column)})")
    
    if total_categories > max_categories:
        print(f"⚠️ Hay demasiadas categorías ({total_categories}), mostrando solo las primeras {max_categories}.")
        print(freq_table.head(max_categories).to_string(index=False))
    else:
        print(freq_table.to_string(index=False))  # Muestra todas las categorías si son pocas
    
    return True  # Indica que esta tabla tenía datos

def show_all_statistics(df, max_categories=100):
    """Muestra estadísticas descriptivas y tablas de frecuencia de todas las columnas.
    
    Retorna:
        - tables_generated: Lista con los códigos de las variables que tuvieron tabla de frecuencia.
        - empty_tables: Lista con los códigos de las variables que estaban vacías.
    """
    tables_generated = []
    empty_tables = []
    
    for col in df.columns:
        if describe_variable(df, col):  # Solo genera la tabla si la variable no está vacía
            if frequency_table(df, col, max_categories):
                tables_generated.append(col)  # Solo el código, sin la etiqueta
            else:
                empty_tables.append(col)
        else:
            empty_tables.append(col)
        
        print("=" * 100)  # Separador visual
    
    print("\n✅ **Resumen de Tablas de Frecuencia:**")
    print("📌 Tablas generadas para los siguientes códigos:")
    print("\n".join(tables_generated) if tables_generated else "❌ No se generaron tablas de frecuencia.")
    
    print("\n⚠️ Variables sin datos (no se generó tabla):")
    print("\n".join(empty_tables) if empty_tables else "✅ Todas las variables tenían datos.")
    
    return tables_generated, empty_tables


# ===========================
# CICLO PARA GRÁFICAS
# ===========================
def plot_variable(df, column, max_categories=20):
    """Grafica la distribución de una variable con una leyenda lateral para mejorar la legibilidad."""
    if column not in df or df[column].isna().all():
        print(f"⚠️ No se puede graficar '{column}', está vacía o no existe.")
        return
    
    if pd.api.types.is_numeric_dtype(df[column]):
        plt.figure(figsize=(8, 4))
        sns.histplot(df[column], bins=20, kde=True, color="royalblue")
        plt.title(f"Distribución de {get_variable_label(df, column)}")
        plt.xlabel(get_variable_label(df, column))
        plt.ylabel("Frecuencia")
        plt.show()
        return
    
    elif isinstance(df[column].dtype, pd.CategoricalDtype) or df[column].dtype == 'object':
        counts = df[column].value_counts().dropna()
        if len(counts) > max_categories:
            counts = counts[:max_categories]
        
        # Convertir los índices a string explícitamente
        categories = counts.index.astype(str)
        
        # Crear la paleta de colores
        palette = sns.color_palette("Blues_r", len(counts))
        
        fig, ax = plt.subplots(figsize=(8, 4))  # Crear figura y eje principal
        bars = ax.bar(range(len(categories)), counts.values, color=palette)
        
        plt.title(f"Frecuencia de {get_variable_label(df, column)}")
        plt.xlabel(get_variable_label(df, column))
        plt.ylabel("Frecuencia")
        plt.xticks([])  # Ocultar los valores en el eje X para evitar amontonamiento
        
        # Crear la leyenda lateral
        legend_labels = [f"{label}: {value}" for label, value in zip(categories, counts.values)]
        patches = [plt.Line2D([0], [0], marker='o', color='w', markerfacecolor=color, markersize=10) 
                   for color in palette]
        
        ax.legend(patches, legend_labels, title="Categorías", loc="center left", bbox_to_anchor=(1, 0.5))
    
    plt.show()

def show_all_graphs(df):
    """Genera gráficos para todas las variables."""
    for col in df.columns:
        print(f"\n📊 Gráfica de {col}")
        plot_variable(df, col)


In [ ]:
tables, empty = show_all_statistics(dataOfTheYear)

print("\n📊 Variables con tabla de frecuencia generada:", tables)
print("\n⚠ Variables sin datos:", empty)


In [ ]:
print(list_filtered_codes_forYear)

In [ ]:
print(empty)
print(len(empty))

In [ ]:
print(tables)
print(len(tables))

In [ ]:
# Códigos que están en list_filtered_codes_forYear pero no en tables
only_in_filtered = set(list_filtered_codes_forYear) - set(tables)

# Códigos que están en tables pero no en list_filtered_codes_forYear
only_in_tables = set(tables) - set(list_filtered_codes_forYear)

# Resultados
print("Códigos solo en list_filtered_codes_forYear:(es decir que faltaron por realizar tabla de frecuencia)", list(only_in_filtered))
print("Cantidad:", len(only_in_filtered))

print("Códigos solo en tables:", list(only_in_tables))
print("Cantidad:", len(only_in_tables))


In [ ]:
def get_labels_for_list(df, lista):
    """Devuelve un diccionario con los códigos y sus etiquetas."""
    return {col: get_variable_label(df, col) for col in lista}

# Ejemplo de uso
labels_dict = get_labels_for_list(df,only_in_filtered)

# Imprimir etiquetas
for code, label in labels_dict.items():
    print(f"{code}: {label}")


In [ ]:
def analyze_variable_frequencies(df, variable_list, max_categories=20):
    """Muestra la frecuencia de variables específicas y detecta si están vacías.
    
    - Si una variable tiene pocas categorías, muestra su tabla de frecuencias.
    - Si está vacía, la lista en una sección aparte.
    
    Retorna:
    - `frequency_data`: Diccionario con la tabla de frecuencias de cada variable no vacía.
    - `empty_variables`: Lista de variables vacías.
    """
    frequency_data = {}
    empty_variables = []

    for var in variable_list:
        if var not in df or df[var].isna().all():
            empty_variables.append(var)
            continue  # Pasa a la siguiente variable si está vacía
        
        # Obtener conteo de valores
        value_counts = df[var].value_counts(dropna=False)
        
        # Si hay pocas categorías, guardar la tabla de frecuencia
        if len(value_counts) <= max_categories:
            freq_table = value_counts.reset_index()
            freq_table.columns = [var, "Frecuencia"]
            freq_table["Porcentaje"] = (freq_table["Frecuencia"] / freq_table["Frecuencia"].sum()) * 100
            frequency_data[var] = freq_table
            print(f"\n📌 **Tabla de Frecuencias para '{var}' ({get_variable_label(df, var)})**")
            print(freq_table.to_string(index=False))
        else:
            print(f"\n⚠️ '{var}' tiene demasiadas categorías ({len(value_counts)}) para mostrar la tabla completa.")

    print("\n⚠️ **Variables sin datos:**")
    if empty_variables:
        print("\n".join(empty_variables))
    else:
        print("✅ Todas las variables tienen datos.")

    return frequency_data, empty_variables

frequency_data, empty_variables = analyze_variable_frequencies(df, only_in_filtered)




In [ ]:
show_all_graphs(dataOfTheYear)